# EXO

### importer les données json dans votre base de données

### combien de bibliotheques se trouve a paris

### Faites un update sur le champs "commune" pour le passer en lower case et refaite le compte des bibliotheques a paris

### afficher tout les noms d'établissements situé 95

### requeter la bdd pour creer un dictionnaire comportant le nombre de bibliotheque ouvert par jour de la semaine

### certains jours sont bien plus representé que d'autres pourquoi ?

# TP

### le TP consiste a :
  - acquerir de la donnees se mettant a jour assez régulièrement via une api ou du scrapping
    ex: (meteo, bourse, crypto-monaie ect.)
  - stocker cette données dans votre base mongodb rajouter la date precise si ce n'est pas dans vos data
  - mettre en place un outils permetant de visualiser ses data par rapport a la date
    ex:(streamlit, flask ect.)